# Try Llama Inference

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import pandas as pd

In [2]:
# model_name = "meta-llama/Llama-2-7b-hf"
model_name = 'meta-llama/Llama-3.2-3B-Instruct'
hf_token = os.getenv('HUGGINGFACE_TOKEN')
hf_token

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", use_auth_token=hf_token)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [4]:
# Test with a sample prompt
input_text = "Why does the Supreme Court hold oral arguments?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=50)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/Users/nnadeem/.virtualenvs/research/lib/python3.12/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(


Why does the Supreme Court hold oral arguments? Oral arguments are a crucial part of the Supreme Court's decision-making process. They provide the Court with an opportunity to hear from the parties involved in a case, their lawyers, and other experts who can


In [5]:
def evaluate_question(question, opening_statement):
    prompt = f"""
    Opening Statement: {opening_statement}

    Question: {question}

    Does the question logically follow from the opening statement? Label it as 'coherent' if it makes sense based on the opening statement alone or 'incoherent' if additional context is needed.

    Label: """
    # inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # use for cluster
    inputs = tokenizer(prompt, return_tensors="pt").to("mps")   # use for mac
    outputs = model.generate(**inputs,max_new_tokens=256)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    label = response.split("Label:")[-1].strip().lower()
    return label

In [6]:
questions_df = pd.read_csv('2024_all_questions_full_text_merged.csv')


In [7]:
questions_df = questions_df[questions_df['transcript_id']  == '2024.23-217-t01']
questions_df = questions_df[questions_df['question_addressee'] == 'petitioner']
questions_df.head()

,transcript_id,question_addressee,justice,question_text,opening_statement,full_text
354,2024.23-217-t01,petitioner,Clarence Thomas,"Other than in the context of actual malice, c...",<speaker>Lisa S. Blatt</speaker><text> Mr. Chi...,"<speaker>John G. Roberts, Jr.</speaker><text> ..."
355,2024.23-217-t01,petitioner,Clarence Thomas,How would you respond -- what do you have to ...,<speaker>Lisa S. Blatt</speaker><text> Mr. Chi...,"<speaker>John G. Roberts, Jr.</speaker><text> ..."
356,2024.23-217-t01,petitioner,"John G. Roberts, Jr.",How are we supposed to -- you make the argum...,<speaker>Lisa S. Blatt</speaker><text> Mr. Chi...,"<speaker>John G. Roberts, Jr.</speaker><text> ..."
357,2024.23-217-t01,petitioner,Brett M. Kavanaugh,"Well, how do we apply the particularly import...",<speaker>Lisa S. Blatt</speaker><text> Mr. Chi...,"<speaker>John G. Roberts, Jr.</speaker><text> ..."
358,2024.23-217-t01,petitioner,Brett M. Kavanaugh,"Are you saying -- and, relatedly, are you sa...",<speaker>Lisa S. Blatt</speaker><text> Mr. Chi...,"<speaker>John G. Roberts, Jr.</speaker><text> ..."


In [8]:
q_sample = questions_df.head(3).copy()
q_sample

,transcript_id,question_addressee,justice,question_text,opening_statement,full_text
354,2024.23-217-t01,petitioner,Clarence Thomas,"Other than in the context of actual malice, c...",<speaker>Lisa S. Blatt</speaker><text> Mr. Chi...,"<speaker>John G. Roberts, Jr.</speaker><text> ..."
355,2024.23-217-t01,petitioner,Clarence Thomas,How would you respond -- what do you have to ...,<speaker>Lisa S. Blatt</speaker><text> Mr. Chi...,"<speaker>John G. Roberts, Jr.</speaker><text> ..."
356,2024.23-217-t01,petitioner,"John G. Roberts, Jr.",How are we supposed to -- you make the argum...,<speaker>Lisa S. Blatt</speaker><text> Mr. Chi...,"<speaker>John G. Roberts, Jr.</speaker><text> ..."


In [ ]:
q_sample['label'] = q_sample.apply(
    lambda row: evaluate_question(row['question_text'], row['opening_statement']), axis=1
)
q_sample